In [25]:
from dash import Dash
from dash import dcc, html, Input, Output, State, callback, ALL, Patch, MATCH, dash_table
import pandas as pd


In [26]:
# read in the metadata file, excluded columns and values
Path = "/Users/lukas-danielf/Documents/Pathologie Marburg/dashtest/test data/metadata_v2.csv"
df = pd.read_csv(Path, sep="\t")

column_list = df.columns.tolist()
excluded_columns = ["ID", "name"]
excluded_values = []

# Create a dictionary with column names as keys and unique values as values
for col in column_list:
    if col not in excluded_columns:
        options_dict = {col: df[col].unique().tolist()}

# Create empty lists to store unique values for each column
unique_lists = [[] for _ in range(len(df.columns) - len(excluded_columns))]


def create_columns_filtered(df, excluded_columns):
    columns_filtered = []
    for x in df.columns:
        if x not in excluded_columns:
            columns_filtered.append(x)
    return columns_filtered

columns_filtered = create_columns_filtered(df, excluded_columns)
print(columns_filtered)

def create_options_dict(df, excluded_columns):
    options_dict = {}
    for col in df.columns:
        if col not in excluded_columns:
            options_dict[col] = df[col].unique().tolist()
    return options_dict

options_dict = create_options_dict(df, excluded_columns)
print(options_dict)


['case', 'tissue type', 'diagnosis', 'tumor type', 'gland size', 'gland count', 'batch', 'groups', 'pan.t_vs_lym.n', 'lym-m_vs_pan-t', 'liv-m_vs_pan-t', 'lym-m_vs_lym-n', 'pan-t_vs_pan-n', 'all', 'global']
{'case': ['ACH 29', 'ACH 50', 'ACH 59', 'ACH 78', 'ACH 88', 'ACH 91', 'ACH 102', 'ACH 105', 'ACH 108', 'ACH 117', 'ACH 118', 'ACH 123', 'ACH 138', 'ACH 161', 'ACH 180', 'ACH 188', 'ACH 226', 'ACH 239', 'ACH 386', 'ACH 392', 'ACH 132', 'ACH 137', 'ACH 190', 'ACH 262', 'ACH 263', 'ACH 463', 'ACH 179', 'ACH 264', 'ACH 483', 'ACH 233', 'ACH 107', 'ACH 110', 'ACH 111', 'ACH 127', 'ACH 131', 'ACH 136', 'ACH 146', 'ACH 151', 'ACH 162', 'ACH 163', 'ACH 173', 'ACH 25', 'ACH 257', 'ACH 265', 'ACH 277', 'ACH 279', 'ACH 28', 'ACH 284', 'ACH 30', 'ACH 301', 'ACH 31'], 'tissue type': ['pancreas', 'lymph node', 'liver'], 'diagnosis': ['tumor', 'normal'], 'tumor type': ['primary', 'non-tumor', 'metastasis', 'relapse'], 'gland size': ['low', nan, 'high'], 'gland count': ['high', nan, 'low'], 'batch':

In [28]:
app = Dash(__name__)

app.layout = html.Div(
    [
        html.Div(children=[
            dcc.Dropdown(
                id="select-columns-dropdown",
                options=columns_filtered,
                value=[],
                multi=True,
            ),
            dcc.Store(id="selectet_columns"),
        ]),
        html.Div(id='container', children=[]),
        html.Br(),
        dcc.Store(id='store', data=[]),
        dash_table.DataTable(df.to_dict('records'), 
                            [{"name": i, "id": i} for i in df.columns], 
                            id='table',
                            page_size=10,
                            style_table={"overflowX": "auto"},),

    ]
)
@app.callback(
    Output("container", "children"),
    Input("select-columns-dropdown", "value"),
    State("container", "children"),
)
def dropdown_children(selected_columns, div_children):
    
    div_children.clear()
    # Iterate over selected columns

    for col in selected_columns:
        options = [{'label': str(option), 'value': str(option)} for option in options_dict.get(col, [])]
        new_child = html.Div(
            dcc.Dropdown(
            id=f"select-{col}-dropdown",
            placeholder=f"Select {col}...",
            options=options,
            multi=True,
        )
        )
        div_children.append(new_child)
    return div_children
#filter dataframe based on selected values
@app.callback(
    Output('store', 'data'),
    Input('select-columns-dropdown', 'value'),
    Input(f'select-{col}-dropdown', 'value'),

)
def store_values(columns, values):
    #create empty dictionary
    filter_dict = {}
    for col in columns:
        for val in values:
            filter_dict[col] = val
    print(filter_dict)
    return filter_dict




In [29]:
if __name__ == "__main__":
    app.run(debug=False, port=8055,)